## Gradient Descient

In [1]:
import random

def f(x):
    return x**2 + 5


def df(x):
    return 2*x


old_x = float('inf')
x = random.randint(0, 10000)
learning_rate = 0.3
epochs = 0


while abs(x - old_x) > 1.0e-7:
    cost = f(x)
    gradx = df(x)
    ## grad takes into account the effect each parameter has on the cost, so 
    ## that's hwo to find the direction of steepest ascent.

    old_x = x
    x -= learning_rate * gradx

    print('EPOCH{}: Cost={:.3f}, x = {:.3f}'.format(epochs, cost, gradx))
    epochs += 1


EPOCH0: Cost=72658581.000, x = 17048.000
EPOCH1: Cost=11625377.160, x = 6819.200
EPOCH2: Cost=1860064.546, x = 2727.680
EPOCH3: Cost=297614.527, x = 1091.072
EPOCH4: Cost=47622.524, x = 436.429
EPOCH5: Cost=7623.804, x = 174.572
EPOCH6: Cost=1224.009, x = 69.829
EPOCH7: Cost=200.041, x = 27.931
EPOCH8: Cost=36.207, x = 11.173
EPOCH9: Cost=9.993, x = 4.469
EPOCH10: Cost=5.799, x = 1.788
EPOCH11: Cost=5.128, x = 0.715
EPOCH12: Cost=5.020, x = 0.286
EPOCH13: Cost=5.003, x = 0.114
EPOCH14: Cost=5.001, x = 0.046
EPOCH15: Cost=5.000, x = 0.018
EPOCH16: Cost=5.000, x = 0.007
EPOCH17: Cost=5.000, x = 0.003
EPOCH18: Cost=5.000, x = 0.001
EPOCH19: Cost=5.000, x = 0.000
EPOCH20: Cost=5.000, x = 0.000
EPOCH21: Cost=5.000, x = 0.000
EPOCH22: Cost=5.000, x = 0.000
EPOCH23: Cost=5.000, x = 0.000
EPOCH24: Cost=5.000, x = 0.000
EPOCH25: Cost=5.000, x = 0.000
EPOCH26: Cost=5.000, x = 0.000
EPOCH27: Cost=5.000, x = 0.000


## Mini Tensorflow For Neural Networks

### Initial Graph Node

In [9]:
import numpy as np


class Node(object):
    def __init__(self, inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []

        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
            # set 'self' node as inbound_nodes's outbound_nodes

        self.value = None

        self.gradients = {}
        # keys are the inputs to this node, and their
        # values are the partials of this node with 
        # respect to that input.
        # \partial{node}{input_i}
        

    def forward(self):
        '''
        Forward propagation. 
        Compute the output value vased on 'inbound_nodes' and store the 
        result in self.value
        '''

        raise NotImplemented
    

    def backward(self):

        raise NotImplemented

In [13]:
class Input(Node):
    def __init__(self):
        '''
        An Input node has no inbound nodes.
        So no need to pass anything to the Node instantiator.
        '''
        Node.__init__(self)

    def forward(self, value=None):
        '''
        Only input node is the node where the value may be passed
        as an argument to forward().
        All other node implementations should get the value of the 
        previous node from self.inbound_nodes
        
        Example: 
        val0: self.inbound_nodes[0].value
        '''
        if value is not None:
            self.value = value
            ## It's is input node, when need to forward, this node initiate self's value.

        # Input subclass just holds a value, such as a data feature or a model parameter(weight/bias)
        
    def backward(self):
        self.gradients = {self:0}
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self] = grad_cost * 1


class CaculatNode(Node):
    def __init__(self, f, *nodes):
        Node.__init__(self, nodes)
        self.func = f

    def forward(self):
        self.value = self.func(map(lambda n: n.value, self.inbound_nodes))


class Add(Node):
    def __init__(self, *nodes):
        Node.__init__(self, nodes)


    def forward(self):
        self.value = sum(map(lambda n: n.value, self.inbound_nodes))
        ## when execute forward, this node caculate value as defined.

class Linear(Node):
    def __init__(self, nodes, weights, bias):
        Node.__init__(self, [nodes, weights, bias])

    def forward(self):
        inbound_nodes = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2].value

        self.value = np.dot(inbound_nodes, weights) + bias
        
    def backward(self):

        # initial a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}

        for n in self.outbound_nodes:
            # Get the partial of the cost w.r.t this node.
            grad_cost = n.gradients[self]

            self.gradients[self.inbound_nodes[0]] = np.dot(grad_cost, self.inbound_nodes[1].value.T)
            self.gradients[self.inbound_nodes[1]] = np.dot(self.inbound_nodes[0].value.T, grad_cost)
            self.gradients[self.inbound_nodes[2]] = np.sum(grad_cost, axis=0, keepdims=False)



class Sigmoid(Node):
    def __init__(self, node):
        Node.__init__(self, [node])


    def _sigmoid(self, x):
        return 1./(1 + np.exp(-1 * x))

    def forward(self):
        self.x = self.inbound_nodes[0].value
        self.value = self._sigmoid(self.x)

    def backward(self):
        self.partial = self._sigmoid(self.x) * (1 - self._sigmoid(self.x))
        
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}

        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]  # Get the partial of the cost with respect to this node.

            self.gradients[self.inbound_nodes[0]] = grad_cost * self.partial
            # use * to keep all the dimension same!.



class MSE(Node):
    def __init__(self, y, a):
        Node.__init__(self, [y, a])


    def forward(self):
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        assert(y.shape == a.shape)

        self.m = self.inbound_nodes[0].value.shape[0]
        self.diff = y - a

        self.value = np.mean(self.diff**2)


    def backward(self):
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff


def forward_and_backward(outputnode, graph):
    # execute all the forward method of sorted_nodes.

    ## In practice, it's common to feed in mutiple data example in each forward pass rather than just 1. Because the examples can be processed in parallel. The number of examples is called batch size.
    for n in graph:
        n.forward()
        ## each node execute forward, get self.value based on the topological sort result.

    for n in  graph[::-1]:
        n.backward()

    #return outputnode.value


def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.
    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.
    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]
            ## if n is Input Node, set n'value as 
            ## feed_dict[n]
            ## else, n's value is caculate as its
            ## inbounds

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def sgd_update(trainables, learning_rate=1e-2):
    # there are so many update methods! 
    # we would talk in next class!
    # like Adam, Mom
    for t in trainables:
        t.value -= learning_rate * t.gradients[t]

In [15]:
"""
Check out the new network architecture and dataset!
Notice that the weights and biases are
generated randomly.
No need to change anything, but feel free to tweak
to test your network, play around with the epochs, batch size, etc!
"""

import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample
#from miniflow import *

# Load data
data = load_boston()
X_ = data['data']
y_ = data['target']

# Normalize data
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

n_features = X_.shape[1]
n_hidden = 10
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Neural network
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

epochs = 3000
# Total number of examples
m = X_.shape[0]
batch_size = 16
steps_per_epoch = m // batch_size

graph = topological_sort(feed_dict)
trainables = [W1, b1, W2, b2]

print("Total number of examples = {}".format(m))

# Step 4
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        # Step 1
        # Randomly sample a batch of examples
        X_batch, y_batch = resample(X_, y_, n_samples=batch_size)

        # Reset value of X and y Inputs
        X.value = X_batch
        y.value = y_batch

        # Step 2
        _ = None
        forward_and_backward(_, graph) # set output node not important.

        # Step 3
        rate = 1e-1
    
        sgd_update(trainables, rate)

        loss += graph[-1].value
    
    if i % 100 == 0: 
        print("Epoch: {}, Loss: {:.3f}".format(i+1, loss/steps_per_epoch))


Total number of examples = 506
Epoch: 1, Loss: 57.318
Epoch: 101, Loss: 5.591
Epoch: 201, Loss: 4.112
Epoch: 301, Loss: 4.249
Epoch: 401, Loss: 3.989
Epoch: 501, Loss: 4.529
Epoch: 601, Loss: 3.882
Epoch: 701, Loss: 3.448
Epoch: 801, Loss: 4.073
Epoch: 901, Loss: 3.507
Epoch: 1001, Loss: 4.170
Epoch: 1101, Loss: 3.817
Epoch: 1201, Loss: 3.851


KeyboardInterrupt: 